In [11]:
# %pip install pandas numpy
# %pip install sentence-transformers
# %pip install scikit-learn
# %pip install matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: numpy 2.2.3
Uninstalling numpy-2.2.3:
  Would remove:
    /Users/godwins/Desktop/Resume/Projects/Delta/.conda/bin/f2py
    /Users/godwins/Desktop/Resume/Projects/Delta/.conda/bin/numpy-config
    /Users/godwins/Desktop/Resume/Projects/Delta/.conda/lib/python3.11/site-packages/numpy-2.2.3.dist-info/*
    /Users/godwins/Desktop/Resume/Projects/Delta/.conda/lib/python3.11/site-packages/numpy/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 32.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninstalled numpy-2.2.3
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sentence_transformers import SentenceTransformer

delta_df = pd.read_csv("datasets/Delta_Airline_Review_Dataset-Asof02172023.csv")
delta_df.head()


,Customer_ID,star rating,date,Seat Type,routes,traveler_types,country,reviews
0,1,4,17th February 2023,Economy Class,New York to Tel Aviv,Solo Leisure,United States,"✅ Trip Verified | First, travel starts 6pm ar..."
1,2,1,16th February 2023,Economy Class,Milwaukee to Ft Lauderdale,Business,United States,✅ Trip Verified | Delta did not issue any wea...
2,3,1,10th February 2023,Economy Class,New York to Charlotte,Business,United States,"✅ Trip Verified | Per our pilot, there would ..."
3,4,1,8th February 2023,Premium Economy,Atlanta to Cape Town,Family Leisure,United States,✅ Trip Verified | We flew from Atlanta to Cap...
4,5,2,6th February 2023,Economy Class,Dallas to Atlanta,Solo Leisure,United Kingdom,✅ Trip Verified | Checkin staff always seem u...


### Data Inspection

In [8]:

print("----Data Info----\n")
print(delta_df.info())

print("\n----Data Shape----\n")
print(delta_df.shape)

print("\n----Data Stats----\n")
print(delta_df.describe())

print("\n----Data Null----\n")
print(delta_df.isnull().sum())


----Data Info----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2689 entries, 0 to 2688
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Customer_ID     2689 non-null   int64 
 1   star rating     2689 non-null   int64 
 2   date            2689 non-null   object
 3    Seat Type      2689 non-null   object
 4   routes          2263 non-null   object
 5   traveler_types  2282 non-null   object
 6   country         2689 non-null   object
 7   reviews         2689 non-null   object
dtypes: int64(2), object(6)
memory usage: 168.2+ KB
None

----Data Shape----

(2689, 8)

----Data Stats----

       Customer_ID  star rating
count  2689.000000  2689.000000
mean   1345.000000     3.860915
std     776.391761     3.462060
min       1.000000     0.000000
25%     673.000000     1.000000
50%    1345.000000     2.000000
75%    2017.000000     8.000000
max    2689.000000    10.000000

----Data Null----

Customer_ID       

### Data Cleaning

In [15]:
def clean_data(df):
    df.dropna(inplace=True)
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].fillna('')
        df[col] = df[col].str.strip().str.lower()
        df['date'] = pd.to_datetime(df['date'])
        df['reviews'] = df['reviews'].str.replace('✅ trip verified |', ' ')
        if df['star rating'] >= 4:
            df['sentiment'] = 'Positive'
        elif df['star rating'] == 3:
            df['sentiment'] = 'Neutral' 
        else:
            df['sentiment'] = 'Negative'
    print("----Data Cleaned----\n")
    print(df.info())
    print(df.isnull().sum())
    return df

clean_data(delta_df)


----Data Cleaned----

<class 'pandas.core.frame.DataFrame'>
Index: 2263 entries, 0 to 2267
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Customer_ID     2263 non-null   int64         
 1   star rating     2263 non-null   int64         
 2   date            2263 non-null   datetime64[ns]
 3    Seat Type      2263 non-null   object        
 4   routes          2263 non-null   object        
 5   traveler_types  2263 non-null   object        
 6   country         2263 non-null   object        
 7   reviews         2263 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 159.1+ KB
None
Customer_ID       0
star rating       0
date              0
 Seat Type        0
routes            0
traveler_types    0
country           0
reviews           0
dtype: int64


,Customer_ID,star rating,date,Seat Type,routes,traveler_types,country,reviews
0,1,4,2023-02-17,economy class,new york to tel aviv,solo leisure,united states,"first, travel starts 6pm arrival at the air..."
1,2,1,2023-02-16,economy class,milwaukee to ft lauderdale,business,united states,delta did not issue any weather waivers so ...
2,3,1,2023-02-10,economy class,new york to charlotte,business,united states,"per our pilot, there would be a slight dela..."
3,4,1,2023-02-08,premium economy,atlanta to cape town,family leisure,united states,we flew from atlanta to cape town and back ...
4,5,2,2023-02-06,economy class,dallas to atlanta,solo leisure,united kingdom,"checkin staff always seem unfriendly, no ma..."
...,...,...,...,...,...,...,...,...
2263,2264,1,2015-07-03,business class,msp to sjc,family leisure,united states,flew from atlanta to los angeles in order to c...
2264,2265,8,2015-07-02,economy class,indianapolis to phoenix via minneapolis,business,united states,the flight was delayed. the boarding process w...
2265,2266,1,2015-07-02,economy class,nrt to nrt via sin,family leisure,united states,many people have had negative reviews about de...
2266,2267,1,2015-07-02,economy class,barcelona to new-york,solo leisure,united states,i often read negative reviews of delta. i for ...


### Data Transformation



In [14]:
def tokenize_text(text):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(text)
    return embeddings

delta_df['embedding'] = delta_df['reviews'].apply(tokenize_text)

delta_df.head()


RuntimeError: Numpy is not available